In [1]:
import pickle

In [4]:
infile = open('pods_open_total','rb')
pods_total = pickle.load(infile)
infile.close()

In [6]:
pods_total.keys()

dict_keys([10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46])